In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.optimize import curve_fit
from astropy import wcs
import glob
from lmfit import Minimizer, Parameters, report_fit

import lmfit
from lmfit.lineshapes import gaussian2d, lorentzian


from matplotlib.ticker import FormatStrFormatter
plt.rcParams['font.size'] = 35
plt.rcParams['axes.labelsize'] = 35
plt.rcParams['xtick.labelsize'] = 35
plt.rcParams['ytick.labelsize'] =35
plt.rcParams['legend.fontsize'] = 35
plt.rcParams['figure.titlesize'] = 12
plt.rcParams['axes.labelweight']='bold'
plt.rcParams['axes.linewidth'] = 3
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['xtick.major.width'] = 5
plt.rcParams['ytick.major.width'] = 3
plt.rcParams['xtick.minor.width'] = 5
plt.rcParams['ytick.minor.width'] = 3

In [ ]:
def residual(params, amp, amperr):
    height = params['height']
    x = params['x']
    y = params['y']
    sigma = params['sigma']
    
    xpix,ypix = np.meshgrid(np.arange(amp.shape[1]),np.arange(amp.shape[0]))
    
    model = height*np.exp(-(((x-xpix)/sigma)**2+((y-ypix)/sigma)**2)/2)
    
    return (amp-model) / amperr

In [ ]:
def gauss_fitter(amp,amperr,plot_gass=False):
    params = Parameters()
    yo_guess, xo_guess = np.unravel_index(np.argmax(amp), amp.shape)
    sigma_x_guess, sigma_y_guess = 2.0, 2.0  # Initial guess for sigma
    params.add('height', value=np.max(amp))
    params.add('x', value=xo_guess, min=xo_guess-0.25, max=xo_guess+0.25) #estimated from looking at the amp values
    params.add('y', value=yo_guess, min=yo_guess-0.25, max=yo_guess+0.25)
    params.add('sigma', value=1,min=0.5, max=2)
    minner = Minimizer(residual, params, fcn_args=(amp, amperr))
    result = minner.minimize()
    
    fit = result.params.valuesdict()
    if plot_gass==True:
        report_fit(result)
        plt.figure(figsize=[12,12])
        xpix,ypix = np.meshgrid(np.linspace(0,amp.shape[0],60),np.linspace(0,amp.shape[1],60))
        bestfit = fit["height"]*np.exp(-(((fit["x"]-xpix)/fit["sigma"])**2+((fit["y"]-ypix)/fit["sigma"])**2)/2)
        plt.contour(xpix,ypix,bestfit,colors='0')
        plt.scatter(xo_guess, yo_guess,c='w',s=200)
        plt.imshow(np.log10(amp/np.max(amp)),origin='lower',vmin=-2,vmax=0.1)
        plt.show()
    return fit["x"],fit["y"]

In [ ]:
def pos_dif(file,j,plot_gass=False):
    cube_hdulist = fits.open(file)

    nw = cube_hdulist[1].shape[0]

    cube_data = cube_hdulist[1].data
    w = wcs.WCS(cube_hdulist[1].header)
    hdr = cube_hdulist[1].header
    wave = (np.arange(nw) + hdr["CRPIX3"]) * hdr["CDELT3"] + hdr["CRVAL3"]


    amp=[]
    xa=[]
    ya=[]
    sigx=[]
    sigy=[]
    ww=[]
    wave=wave[0:np.shape(cube_data)[0]]


    i=0
    while i<np.shape(cube_data)[0]:
        # Fit Gaussian to the central region of the cube
        z_index = i#np.shape(cube_data)[0] // 2  # Choose central spectral channel
        amp = np.mean(cube_data[z_index:z_index+j],axis=0)
        amperr=np.std(cube_data[z_index:z_index+j],axis=0)
        amp[np.isnan(amp)]=1e-10
        amp[amp<np.max(amp)*0.05]=1e-10
        amperr[np.isnan(amperr)]=1e-10
        x1,y1=gauss_fitter(amp,amperr,plot_gass)
        a=w.pixel_to_world(x1,y1,0)[0]
        xa.append((a.ra).deg)
        ya.append((a.dec).deg)
        ww.append(np.mean(wave[z_index:z_index+j]))
        i=i+(j//3)
    xa=np.array(xa)
    ya=np.array(ya)
    ww=np.array(ww)

    x1=np.percentile(xa,40)
    x2=np.percentile(xa,60)

    ya=ya[(x1<xa) & (xa<x2)]
    ww=ww[(x1<xa) & (xa<x2)]
    xa=xa[(x1<xa) & (xa<x2)]

    plt.figure(figsize=[12,12])
    plt.scatter(ww,(xa-np.mean(xa))*3600,label='RA',s=100,c='b')
    plt.scatter(ww,(ya-np.mean(ya))*3600,label='Dec',s=100,c='orange')
    plt.legend()
    plt.ylim(-0.1,0.1)
    plt.xlabel('Wavelength ($\mu$m)')
    plt.ylabel('Offset from mean (arcsec)')
        
        
    print("In "+ str(file) + " the mean RA is " + str(np.mean(xa))+ " degs and mean Dec is "+ str(np.mean(ya))+ ' degs with mean\
          wavelength in mircons '+ str(np.mean(ww)))
    return np.mean(xa),np.mean(ya),np.mean(ww)

       

In [ ]:
xa,ya,ww=pos_dif('Level3_ch4-short_s3d.fits',60,plot_gass=True)




In [ ]:
files=glob.glob('*fits*')
ra=[]
dec=[]
ww=[]
for file in files:
    print(file)
    ra1,dec1,ww1=pos_dif(file,60)
    plt.plot(ww1,ra1)
    ra.append(ra1)
    dec.append(dec1)
    ww.append(ww1)


In [ ]:
(ra -np.median(ra))*3600.0

In [ ]:
(dec -np.median(dec))*3600.0


In [ ]:
offset=(((ra -np.median(ra))*3600.0)**2 + ((dec -np.median(dec))*3600.0)**2)**0.5

In [ ]:
plt.figure(figsize=[12,12])
plt.scatter(ww,offset,s=100,c='b')
plt.legend()
plt.ylim(-1,1)
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('Offset from mean (arcsec)')

In [ ]:
offset
ww=np.array(ww)

In [ ]:
inds = ww.argsort()
offset= offset[inds]
ww.sort()

In [ ]:
ww

In [ ]:
offset